# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


In [1]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers.core import Dense,Dropout,Activation,Flatten
from keras.layers.convolutional import Conv2D,MaxPooling2D
from keras.utils import np_utils
from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [2]:
from keras.datasets import cifar10

In [3]:
#Load the dataset:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [4]:
X_train.shape

(50000, 32, 32, 3)

In [5]:
X_test.shape

(10000, 32, 32, 3)

In [6]:
X1_train = []
X1_test = []
X2_train = []
X2_test = []
Y1_train = []
Y1_test = []
Y2_train = []
Y2_test = []

In [7]:
totalImageInTrainning = X_train.shape[0]
maxIntensity = X_train[0].max()

In [8]:
#split the trainning set
for item in range(totalImageInTrainning):
    if y_train[item]<5:
        X1_train.append(X_train[item]/maxIntensity)
        Y1_train.append(y_train[item])
    else:
        X2_train.append(X_train[item]/maxIntensity)
        Y2_train.append(y_train[item])

In [9]:
totalImageInTestset = X_test.shape[0]
maxTestIntensity = X_test[0].max()

In [10]:
#split the test set
for item in range(totalImageInTestset):
    if y_test[item]<5:
        X1_test.append(X_test[item]/maxTestIntensity)
        Y1_test.append(y_test[item])
    else:
        X2_test.append(X_test[item]/maxTestIntensity)
        Y2_test.append(y_test[item])

In [11]:
#reshape the array to 32,32,3
X1_train = np.asarray(X1_train).reshape((-1, 32, 32, 3))
X1_test = np.asarray(X1_test).reshape((-1, 32, 32, 3))
X2_train = np.asarray(X2_train).reshape((-1, 32, 32, 3))
X2_test = np.asarray(X2_test).reshape((-1, 32, 32, 3))

In [12]:
print(X1_train.shape)
print(X1_test.shape)
print(X2_train.shape)
print(X2_test.shape)

(25000, 32, 32, 3)
(5000, 32, 32, 3)
(25000, 32, 32, 3)
(5000, 32, 32, 3)


In [13]:
print('First set lable range', min(Y1_train),max(Y1_train))

First set lable range [0] [4]


In [14]:
print('Second set lable range', min(Y2_train),max(Y2_train))

Second set lable range [5] [9]


### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [15]:
#One-hot encoding the labels
num_classes = 5
Y1_train = np_utils.to_categorical(np.asarray(Y1_train), num_classes)
Y1_test = np_utils.to_categorical(np.asarray(Y1_test), num_classes)

In [16]:
Y1_train.shape

(25000, 5)

In [17]:
num_classes = 10
Y2_train = np_utils.to_categorical(np.asarray(Y2_train), num_classes)
Y2_test = np_utils.to_categorical(np.asarray(Y2_test), num_classes)

In [18]:
Y2_train.shape

(25000, 10)

In [19]:
Y2_train.max()

1.0

In [20]:
Y1_test.shape

(5000, 5)

In [21]:
Y2_test.shape

(5000, 10)

In [22]:
Y1_train[0]

array([0., 0., 0., 0., 1.], dtype=float32)

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [23]:
X1_train.shape[1:]

(32, 32, 3)

In [24]:
import time, datetime

In [25]:
#Building up a Sequential model
from keras.layers import Dropout, Flatten, GlobalAveragePooling2D

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu',input_shape = X1_train.shape[1:]))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(GlobalAveragePooling2D())
model.add(Dense(1000, activation='relu')) 
model.add(Dense(5, activation='softmax'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 64)          18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 2, 2, 64)          0         
_________________________________________________________________
glob

In [26]:
model.compile(loss='binary_crossentropy', optimizer='adam',
              metrics=['accuracy'])

In [27]:
batch_size=32
epochs = 10
start = datetime.datetime.now()
model.fit(X1_train, Y1_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X1_test, Y1_test))
time_taken = datetime.datetime.now() - start
print('Time Taken for Trainning:',time_taken)

Instructions for updating:
Use tf.cast instead.
Train on 25000 samples, validate on 5000 samples
Epoch 1/10
25000/25000 [==============================] - 31s 1ms/step - loss: 0.3755 - acc: 0.8322 - val_loss: 0.3238 - val_acc: 0.8557
Epoch 2/10
25000/25000 [==============================] - 30s 1ms/step - loss: 0.3034 - acc: 0.8665 - val_loss: 0.2834 - val_acc: 0.8782
Epoch 3/10
25000/25000 [==============================] - 33s 1ms/step - loss: 0.2722 - acc: 0.8824 - val_loss: 0.2766 - val_acc: 0.8796
Epoch 4/10
25000/25000 [==============================] - 32s 1ms/step - loss: 0.2447 - acc: 0.8970 - val_loss: 0.2669 - val_acc: 0.8852
Epoch 5/10
25000/25000 [==============================] - 34s 1ms/step - loss: 0.2255 - acc: 0.9056 - val_loss: 0.2353 - val_acc: 0.8998
Epoch 6/10
25000/25000 [==============================] - 36s 1ms/step - loss: 0.2095 - acc: 0.9128 - val_loss: 0.2367 - val_acc: 0.9014
Epoch 7/10
25000/25000 [==============================] - 36s 1ms/step - loss: 0.

In [28]:
#Evaluate the model on the test data
score = model.evaluate(X1_test, Y1_test)

#Accuracy on test data
print('Accuracy on the Test Images: ', score[1])

5000/5000 [==============================] - 3s 570us/step
Accuracy on the Test Images:  0.9128399965286255


### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [29]:
model.layers

In [30]:
for layer in model.layers[:7]:
    layer.trainable = False

### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [31]:
transfer_model = Sequential(model.layers[:7])
transfer_model.add(Dense(1000, activation='relu')) 
transfer_model.add(Dense(10, activation='softmax'))
transfer_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 64)          18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 2, 2, 64)          0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 64)                0         
__________

In [32]:
transfer_model.compile(loss='binary_crossentropy', optimizer='adam',
              metrics=['accuracy'])

In [33]:
batch_size=32
epochs = 10
start = datetime.datetime.now()
transfer_model.fit(X2_train, Y2_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X2_test, Y2_test))
time_taken = datetime.datetime.now() - start
print('Time Taken for Trainning:',time_taken)

Train on 25000 samples, validate on 5000 samples
Epoch 1/10
25000/25000 [==============================] - 21s 844us/step - loss: 0.1405 - acc: 0.9412 - val_loss: 0.1271 - val_acc: 0.9473
Epoch 2/10
25000/25000 [==============================] - 20s 803us/step - loss: 0.1237 - acc: 0.9491 - val_loss: 0.1230 - val_acc: 0.9490
Epoch 3/10
25000/25000 [==============================] - 20s 802us/step - loss: 0.1186 - acc: 0.9510 - val_loss: 0.1221 - val_acc: 0.9489
Epoch 4/10
25000/25000 [==============================] - 22s 868us/step - loss: 0.1142 - acc: 0.9525 - val_loss: 0.1159 - val_acc: 0.9517
Epoch 5/10
25000/25000 [==============================] - 19s 755us/step - loss: 0.1105 - acc: 0.9541 - val_loss: 0.1144 - val_acc: 0.9524
Epoch 6/10
25000/25000 [==============================] - 21s 830us/step - loss: 0.1074 - acc: 0.9555 - val_loss: 0.1137 - val_acc: 0.9531
Epoch 7/10
25000/25000 [==============================] - 20s 793us/step - loss: 0.1055 - acc: 0.9560 - val_loss: 0.1

In [34]:
#Evaluate the model on the test data
score1 = transfer_model.evaluate(X2_test, Y2_test)

#Accuracy on test data
print('Accuracy on the Test Images: ', score1[1])

5000/5000 [==============================] - 3s 626us/step
Accuracy on the Test Images:  0.9551200227737426


We could see that for the first set of learning for lable 0-4, the trainning time took 5:35 and the accuracy was arround 91%. Using transfer learning we could achive faster learning in 03:21 minutes and the accuracy is 95%. This shows transfer learning is very usefull for computation reduction.

# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [67]:
from sklearn.datasets import fetch_20newsgroups

In [68]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [37]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [38]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [39]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2], dtype=int64)

In [40]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [41]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [42]:
#Preprocess the text
import unicodedata
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [43]:
def preprocess(text, remove_digits=False):
    try:
        pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
        removeSlash = r'[\n\r\t]'
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        text = re.sub(pattern, '', text)
        text = re.sub(removeSlash,' ',text)
        text = re.sub(' +', ' ', text).strip()
        text = text.lower()
        return text;
    except Exception as e:
        return ""

In [44]:
for index in range(len(twenty_train.data)):
    twenty_train.data[index] = preprocess(twenty_train.data[index],True)

In [45]:
for index in range(len(twenty_test.data)):
    twenty_test.data[index] = preprocess(twenty_test.data[index],True)

In [46]:
# create the transform
Vectorizer = TfidfVectorizer()

In [47]:
Vectorizer.fit(twenty_train.data)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [48]:
print('Train IDF Features',Vectorizer.idf_)

Train IDF Features [5.54418091 5.42639788 5.25649884 ... 7.62362246 8.02908756 8.02908756]


In [49]:
Vectorizer.fit(twenty_test.data)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [50]:
print('Test IDF Features',Vectorizer.idf_)

Test IDF Features [4.73169945 5.31948612 5.05712185 ... 7.62207121 6.92892403 7.62207121]


In [51]:
Vector = Vectorizer.transform(twenty_train.data)

In [52]:
Vector.shape

(2257, 30672)

In [53]:
TestVector = Vectorizer.transform(twenty_test.data)

In [54]:
TestVector.shape

(1502, 30672)

### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [55]:
x_names = Vectorizer.get_feature_names()
x_names

['__',
 '___',
 '____',
 '_____',
 '______',
 '_______',
 '________',
 '_________',
 '__________',
 '____________',
 '_____________',
 '______________',
 '_______________',
 '________________',
 '__________________',
 '_____________________',
 '_______________________',
 '___________________________',
 '____________________________',
 '_____________________________',
 '______________________________________',
 '________________________________________________________',
 '__________________________________________________________',
 '___________________________________________________________',
 '_____________________________________________________________',
 '_______________________________________________________________',
 '________________________________________________________________',
 '__________________________________________________________________',
 '_____________________________________________________________________',
 '_________________________________________________

In [56]:
x_TrainVector = pd.DataFrame(Vector.toarray(),columns=x_names)

In [57]:
x_TrainVector.shape

(2257, 30672)

In [58]:
x_TestVector = pd.DataFrame(TestVector.toarray(),columns=x_names)

In [59]:
x_TestVector.shape

(1502, 30672)

In [60]:
y_train = twenty_train.target

In [61]:
y_test = twenty_test.target

In [62]:
from sklearn.linear_model import LogisticRegression

In [63]:
LRmodel = LogisticRegression()

In [64]:
LRmodel.fit(x_TrainVector, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [65]:
print('Training accuracy:',LRmodel.score(x_TrainVector, y_train))

Training accuracy: 0.9818342933097032


In [66]:
print('Test accuracy:',LRmodel.score(x_TestVector,y_test))

Test accuracy: 0.8621837549933422
